In [1]:
import numpy as np
import pandas as pd
import os.path
import librosa
import IPython.display as ipd
from shutil import copyfile
import glob
import matplotlib.style as ms
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle
import random
import time

import IPython.display
import librosa.display
import joblib
from joblib import Parallel, delayed
from PIL import Image
import multiprocessing as mp

In [2]:
sr = 22050

In [3]:
import pandas as pd
import math

labels_df = pd.read_csv('/home/arpitsah/Desktop/Projects Fall-22/DA/domain_adaptation/LSTM-DENSE/speech-emotion-recognition-iemocap/preprocess_info/df_iemocap.csv')
iemocap_dir = '/home/arpitsah/Desktop/Projects Fall-22/DA/domain_adaptation/LSTM-DENSE/speech-emotion-recognition-iemocap/data/IEMOCAP_full_release_FILES/IEMOCAP_full_release/'
save_dir = '/home/arpitsah/Desktop/Projects Fall-22/DA/domain_adaptation/LSTM-DENSE/speech-emotion-recognition-iemocap/preprocess_info/'
audio_vectors_path= save_dir + 'audio_vectors_'

In [4]:
labels_df.emotion.value_counts()

xxx    2021
fru    1473
neu    1329
ang     936
sad     869
exc     806
hap     454
sur      91
fea      30
oth       3
dis       2
Name: emotion, dtype: int64

In [5]:
emotion_dict = {'ang': 0,
                'hap': 1,
                'exc': 2,
                'sad': 3,
                'fru': 4,
                'fea': 5,
                'sur': 6,
                'neu': 7,
                'xxx': 8,
                'oth': 8,
                'dis': 8}

In [6]:
emotion_full_dict = {'neu': 'neutral', 'ang': 'anger', 'hap': 'happiness', 'exc': 'excited', 'sad': 'sadness',
                     'fru':'frustrated', 'fea': 'fear', 'sur': 'surprised', 'xxx': 'others', 'oth': 'others', 'dis': 'others'}

In [7]:
labels_df.shape

(8014, 7)

In [8]:
labels_df.head()

,start_time,end_time,wav_file,emotion,val,act,dom
0,5.3913,8.2350,Ses01M_script01_2_F000,ang,2.5,2.5,3.0
1,12.7400,17.2800,Ses01M_script01_2_F001,ang,2.5,2.0,3.0
2,32.6300,35.3425,Ses01M_script01_2_F002,xxx,2.5,2.5,3.0
3,36.7600,39.4800,Ses01M_script01_2_F003,xxx,2.5,3.0,4.0
4,41.1800,44.7000,Ses01M_script01_2_F004,ang,1.5,4.0,4.0


In [9]:
labels_df[labels_df["wav_file"].str.contains("Ses01")].shape

(1819, 7)

In [10]:
labels_df[labels_df["wav_file"].str.contains("Ses02")].shape

(1811, 7)

In [11]:
labels_df[labels_df["wav_file"].str.contains("Ses03")].shape

(2136, 7)

In [12]:
labels_df[labels_df["wav_file"].str.contains("Ses04")].shape

(2103, 7)

In [13]:
labels_df[labels_df["wav_file"].str.contains("Ses05")].shape

(145, 7)

In [14]:
labels_df_subset = labels_df[labels_df["emotion"].isin(["neu", 'ang', 'hap', 'sad'])]

In [15]:
labels_df_subset.emotion.value_counts()

neu    1329
ang     936
sad     869
hap     454
Name: emotion, dtype: int64

In [16]:
labels_df_subset = labels_df_subset[["wav_file", "emotion"]]

In [17]:
labels_df_subset = labels_df_subset.rename(columns={"wav_file": "filename"})

In [18]:
labels_df_subset.head()

,filename,emotion
0,Ses01M_script01_2_F000,ang
1,Ses01M_script01_2_F001,ang
4,Ses01M_script01_2_F004,ang
5,Ses01M_script01_2_F005,ang
9,Ses01M_script01_2_F009,ang


In [19]:
pickle_to_df = pd.DataFrame(columns=["filename", "audio_vector"])

In [20]:
for sess in range(1, 6):
    audio_vectors = pickle.load(open('{}{}.pkl'.format(audio_vectors_path, sess), 'rb'))
    for index, row in tqdm(labels_df[labels_df['wav_file'].str.contains('Ses0{}'.format(sess))].iterrows()):

        wav_file_name = row['wav_file']
        y = audio_vectors[wav_file_name]
        list_y = list(y)
        pickle_to_df = pickle_to_df.append({'filename': wav_file_name, 'audio_vector': list_y}, ignore_index=True)

0it [00:00, ?it/s]/tmp/ipykernel_574125/2879380524.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pickle_to_df = pickle_to_df.append({'filename': wav_file_name, 'audio_vector': list_y}, ignore_index=True)
/tmp/ipykernel_574125/2879380524.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pickle_to_df = pickle_to_df.append({'filename': wav_file_name, 'audio_vector': list_y}, ignore_index=True)
/tmp/ipykernel_574125/2879380524.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pickle_to_df = pickle_to_df.append({'filename': wav_file_name, 'audio_vector': list_y}, ignore_index=True)
/tmp/ipykernel_574125/2879380524.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

In [21]:
pickle_to_df.to_csv('/home/arpitsah/Desktop/Projects Fall-22/DA/domain_adaptation/LSTM-DENSE/speech-emotion-recognition-iemocap/preprocess_info/pickle_to_audiodf.csv')

In [22]:
pickle_to_df.shape

(8014, 2)

In [23]:
pickle_to_df.head()

,filename,audio_vector
0,Ses01M_script01_2_F000,"[-0.00021913367, -0.00019357128, -0.0001968474..."
1,Ses01M_script01_2_F001,"[0.003000468, 0.0026301022, 0.0024772626, 0.00..."
2,Ses01M_script01_2_F002,"[0.00058673247, 0.00010428111, 7.208043e-05, 0..."
3,Ses01M_script01_2_F003,"[-0.011382388, -0.012786157, -0.01342686, -0.0..."
4,Ses01M_script01_2_F004,"[-0.0010822626, -0.001223637, -0.00093686994, ..."


In [24]:
pickle_to_subset = labels_df_subset.merge(pickle_to_df, on="filename")

In [25]:
print(pickle_to_subset.shape)
print(pickle_to_df.shape)
print(labels_df_subset.shape)

(3588, 3)
(8014, 2)
(3588, 2)


In [26]:
pickle_to_subset.head()

,filename,emotion,audio_vector
0,Ses01M_script01_2_F000,ang,"[-0.00021913367, -0.00019357128, -0.0001968474..."
1,Ses01M_script01_2_F001,ang,"[0.003000468, 0.0026301022, 0.0024772626, 0.00..."
2,Ses01M_script01_2_F004,ang,"[-0.0010822626, -0.001223637, -0.00093686994, ..."
3,Ses01M_script01_2_F005,ang,"[-0.00034929565, 0.0030486924, -0.007080719, -..."
4,Ses01M_script01_2_F009,ang,"[-0.016453937, -0.015856033, -0.014984095, -0...."


In [27]:
pathAudio = iemocap_dir
pathImage = '/home/arpitsah/Desktop/Projects Fall-22/DA/domain_adaptation/LSTM-DENSE/speech-emotion-recognition-iemocap/data/IEMOCAP_full_release_FILES/IEMOCAP_full_release/IEMOCAP_Image/'

In [28]:
sr = 22050
start = time.time()
def save_spectrogram(y, image_fname, label):
    try:
        start1 = time.time()
        print("inside function")
        S = librosa.feature.melspectrogram(np.asarray(y), sr=sr, n_mels=128)
        log_S = librosa.power_to_db(S, ref=np.max)
        librosa.display.specshow(log_S, sr=sr, x_axis='time', y_axis='mel')
        fig1 = plt.gcf()
        plt.axis('off')
        plt.draw()
        save_path = pathImage + emotion_full_dict[label] + '/' + image_fname + '.jpg'
        fig1.savefig(save_path, dpi=50)
        end1 =  time.time()
        print(end1 - start1)
    except:
        print("no spectrogram was saved for file {}".format(image_fname))
    
pool = mp.Pool(mp.cpu_count())
pool_starmap = pool.starmap_async(save_spectrogram,[(getattr(row, "audio_vector"), getattr(row, "filename"), getattr(row, "emotion")) for row in pickle_to_subset.itertuples(index=True, name='Pandas')])
pool_starmap.wait()
pool.close()
end = time.time()
print(end-start)


inside function
inside function
no spectrogram was saved for file Ses01M_script01_2_F000no spectrogram was saved for file Ses01M_script01_3_F025

inside functioninside function

no spectrogram was saved for file Ses01M_script01_3_F026
inside function
no spectrogram was saved for file Ses01M_script01_2_F001
inside function
no spectrogram was saved for file Ses01M_script01_3_F030
inside functionno spectrogram was saved for file Ses01M_script01_2_F004

inside function
no spectrogram was saved for file Ses01M_script01_3_M000
inside function
no spectrogram was saved for file Ses01M_script01_2_F005
inside function
no spectrogram was saved for file Ses01M_script01_3_M001
inside function
no spectrogram was saved for file Ses01M_script01_3_M002
inside function
no spectrogram was saved for file Ses01M_script01_2_F009
no spectrogram was saved for file Ses01M_script01_3_M004
inside functioninside function

no spectrogram was saved for file Ses01M_script01_2_F011
inside function
no spectrogram was 

In [ ]:
S = librosa.feature.melspectrogram(np.asarray(y), sr=sr, n_mels=128)
log_S = librosa.power_to_db(S, ref=np.max)
librosa.display.specshow(log_S, sr=sr, x_axis='time', y_axis='mel')

In [ ]:
save_path = pathImage + "neu" + '/' + image_fname + '.jpg'

In [ ]:
from PIL import Image
im=Image.open('/home/user/shaheen.perveen/SER_new/IEMOCAP_image/neutral/Ses05F_impro05_M010.jpg')
im.size


In [ ]:
plt.imshow(im)
plt.show()

In [ ]:
im_resize = im.resize((256, 256), Image.ANTIALIAS)

In [ ]:
plt.imshow(im_resize)
plt.show()